In [1]:
# IMPORTAR LIBRERIAS
import os #trabajar con el sistema operativo y variables de entorno
from dotenv import load_dotenv #cargar variable de entorno
load_dotenv() #carga las variables del entorno .env; devuelve un true o false
import spotipy #librería para trabajar con la API de Spotify
from spotipy.oauth2 import SpotifyClientCredentials #autenticación con Spotify

import pandas as pd #dataframes
import requests #peticiones a APIs
import time #librería para hacer pausas dentro de las peticiones y no saturar APIs
from urllib.parse import quote #esta librería sirve para codificar las URLs (espacios y caracteres especiales -> %20, etc)

In [2]:
#AUTENTICACION EN SPOTIFY PARA EXTRAER DATOS DE API

mis_credenciales = SpotifyClientCredentials(
    client_id=os.getenv("SPOTIFY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIFY_CLIENT_SECRET")
)
spotify = spotipy.Spotify(auth_manager=mis_credenciales)

In [3]:
#EXTRAER DATOS DE CANCIONES DE SPOTIFY
generos = ["flamenco", "latin", "jazz", "rock"]
año = 2014
todas_las_canciones = []
albumes_ya_vistos = set()  # Evita duplicados: un álbum puede aparecer al buscar varios artistas (ej. colaboraciones).
datos_artistas = {}
datos_artistas = {} 
artistas_procesados = set()

print("Buscando canciones del año", año)

for genero in generos:
    print("Género:", genero)

    # 👉 ANOTACIÓN: "genre:" SOLO funciona al buscar artistas (type="artist")
    #    No funciona con álbumes ni canciones directamente.
    resultado_artistas = spotify.search(q="genre:" + genero, type="artist", limit=50)
    artistas = resultado_artistas["artists"]["items"]

    for artista in artistas:
        nombre_artista = artista["name"]

        # 👉 ANOTACIÓN: "artist:" y "year:" SÍ se pueden usar juntos
        #    al buscar álbumes (type="album") o canciones (type="track").
        #    Aquí buscamos álbumes del artista en el año indicado.
        busqueda = "artist:" + nombre_artista + " year:" + str(año)
        resultado_albumes = spotify.search(q=busqueda, type="album", limit=50)
        albumes = resultado_albumes["albums"]["items"]
        



        for album in albumes:
            id_album = album["id"]
            nombre_album = album["name"]
            
            # Si ya vimos este álbum, saltamos al siguiente
            if id_album in albumes_ya_vistos:
                continue
            
            # Si no lo habíamos visto, lo marcamos como visto
            albumes_ya_vistos.add(id_album)

            # Pedimos las canciones del álbum (esto no usa anotaciones, es una llamada directa)
            resultado_canciones = spotify.album_tracks(id_album)
            canciones = resultado_canciones["items"]

            for cancion in canciones:
                info = {
                    "nombre": cancion["name"],
                    "artista": nombre_artista,
                    "album": nombre_album,
                    "genero": genero,
                    "año": año
                }
                todas_las_canciones.append(info)

    print("  → Canciones de este género añadidas")
    print()

# Mostramos un resumen de lo encontrado
print("RESUMEN:")

# Contamos las canciones de cada género una por una
for genero in generos:
    contador = 0
    for cancion in todas_las_canciones:
        if cancion["genero"] == genero:
            contador = contador + 1
    print(f"- {genero}: {contador} canciones")

print("Total de canciones encontradas:", len(todas_las_canciones))

Buscando canciones del año 2014
Género: flamenco
  → Canciones de este género añadidas

Género: latin
  → Canciones de este género añadidas

Género: jazz
  → Canciones de este género añadidas

Género: rock
  → Canciones de este género añadidas

RESUMEN:
- flamenco: 6 canciones
- latin: 2806 canciones
- jazz: 5727 canciones
- rock: 3694 canciones
Total de canciones encontradas: 12233


In [4]:
# PASAR LA LISTA DE DATOS A EXTRAIDOS A UN DATAFRAME CON PANDAS
todas_las_canciones_df = pd.DataFrame(todas_las_canciones)
todas_las_canciones_df

,nombre,artista,album,genero,año
0,Life,LaRumbé,Sonido Obscuro,flamenco,2014
1,Boomerang,LaRumbé,Sonido Obscuro,flamenco,2014
2,Getaway Car,LaRumbé,Sonido Obscuro,flamenco,2014
3,Falling,LaRumbé,Sonido Obscuro,flamenco,2014
4,Dullness,LaRumbé,Sonido Obscuro,flamenco,2014
...,...,...,...,...,...
12228,By the Way,Red Hot Chili Peppers,The Studio Album Collection 1991 - 2011,rock,2014
12229,Universally Speaking,Red Hot Chili Peppers,The Studio Album Collection 1991 - 2011,rock,2014
12230,This Is the Place,Red Hot Chili Peppers,The Studio Album Collection 1991 - 2011,rock,2014
12231,Dosed,Red Hot Chili Peppers,The Studio Album Collection 1991 - 2011,rock,2014


In [5]:
# PASAR DATAFRAME A UN ARCHIVO CSV
todas_las_canciones_df.to_csv("canciones_2014.csv", index=False)


In [6]:
# AUTENTICACION EN LASTFM
api_key_lastfm = os.getenv("API_KEY_LASTFM")
shared_secret_lastfm = os.getenv("SHARED_SECRET_LASTFM")

In [7]:
if not api_key_lastfm:
    print("ERROR: La variable de entorno 'API_KEY_LASTFM' no está configurada.")
else:
    print("API Key de Last.fm cargada con éxito.")

API Key de Last.fm cargada con éxito.
